In [1]:
!pip install pyter3 sacrebleu

In [2]:
import pandas as pd

csv_filename = "results_of_nllb-200-distilled-600M.csv"
# csv_original = "results_of_zh2ko_based_on_nllb-200-distilled-600M.csv"
df = pd.read_csv(csv_filename)
df.columns

Index(['source_texts', 'target_texts', 'references'], dtype='object')

In [3]:
source_texts = df["source_texts"]
target_texts = df["target_texts"]
references = df["references"]

BLEU score:

In [4]:
from konlpy.tag import Okt
# from konlpy.tag import Mecab # Warning:KoNLPy’s Mecab() class is not supported on Windows machines.
from nltk.translate.bleu_score import corpus_bleu

# corpus_bleu 的结果更稳健，因为它是基于整个语料库的平均分数计算的，而 sentence_bleu 仅基于单个句子。
# 注：nltk3.8.1和python 3.12有点问题，要么降低python版本，要么按照以下链接的指导去修改bleu_score.py文件
# https://github.com/nltk/nltk/pull/3207
# https://github.com/nltk/nltk/blob/develop/nltk/translate/bleu_score.py
references_okt = [[Okt().morphs(reference)] for reference in references]
target_texts_okt = [Okt().morphs(target_text) for target_text in target_texts]
bleu_score = corpus_bleu(references_okt, target_texts_okt)
bleu_score

0.06025297433806512

METEOR score:

In [5]:
from nltk.translate.meteor_score import meteor_score
from tqdm.notebook import tqdm

meteor_scores = []
for reference, target_text in tqdm(zip(references_okt, target_texts_okt), total=len(references)):
    meteor_scores.append(meteor_score(reference, target_text))
meteor_scores

  0%|          | 0/15000 [00:00<?, ?it/s]

[0.6058098915241772,
 0.3661542923433875,
 0.541907514450867,
 0.551980198019802,
 0.5324644225506294,
 0.1755983350676379,
 0.45784333596687526,
 0.0859106529209622,
 0.4499251147830784,
 0.49646925717852686,
 0.5182909090909091,
 0.34519240396489004,
 0.3409787942516649,
 0.38917089678510997,
 0.6435810810810811,
 0.4148325997065493,
 0.48191193363776164,
 0.43684710351377015,
 0.4775065104166667,
 0.2297454098696335,
 0.7273598977200084,
 0.21609791683553534,
 0.4707098440091846,
 0.4865005492281898,
 0.3960634920634921,
 0.4510080850846766,
 0.3416584158415842,
 0.25969226634415105,
 0.44217808493017674,
 0.4984662576687116,
 0.5868421052631579,
 0.6373344935232697,
 0.5873593151340997,
 0.41838649155722324,
 0.611317254174397,
 0.47405886521866014,
 0.492718267169365,
 0.45406622498935717,
 0.14150362987572293,
 0.2272512832125604,
 0.5835741178530777,
 0.40782577121025543,
 0.5287742382271468,
 0.579569458250497,
 0.3560787276008225,
 0.3289473684210526,
 0.5426045016077171,
 0.2

In [6]:
average_meteor_score = sum(meteor_scores) / len(meteor_scores)
average_meteor_score

0.4119861187858015

TER score:

In [7]:
import pyter

ter_scores = []
for reference, target_text in tqdm(zip(references_okt, target_texts_okt), total=len(references)):
    ter_scores.append(pyter.ter(target_text, reference[0]))
ter_scores

  0%|          | 0/15000 [00:00<?, ?it/s]

[0.5454545454545454,
 0.6511627906976745,
 0.6470588235294118,
 0.625,
 0.6,
 18.02857142857143,
 0.7333333333333333,
 13.076923076923077,
 0.625,
 0.6136363636363636,
 0.4772727272727273,
 0.7317073170731707,
 0.9354838709677419,
 0.8205128205128205,
 0.3783783783783784,
 0.8,
 0.6415094339622641,
 0.782608695652174,
 0.6578947368421053,
 13.78125,
 0.2692307692307692,
 0.8888888888888888,
 0.5588235294117647,
 0.5714285714285714,
 0.6571428571428571,
 0.717391304347826,
 0.9591836734693877,
 8.95,
 0.5897435897435898,
 0.4897959183673469,
 0.5333333333333333,
 0.43478260869565216,
 0.46153846153846156,
 2.883720930232558,
 0.45454545454545453,
 0.6363636363636364,
 0.875,
 4.833333333333333,
 12.261904761904763,
 1.0,
 0.5,
 0.6923076923076923,
 0.5,
 0.58,
 0.7142857142857143,
 0.8666666666666667,
 0.5483870967741935,
 10.83673469387755,
 6.6,
 0.6176470588235294,
 0.6571428571428571,
 0.7948717948717948,
 0.5227272727272727,
 0.3488372093023256,
 0.5581395348837209,
 0.558139534883

In [8]:
average_ter_score = sum(ter_scores) / len(ter_scores)
average_ter_score

3.1401320946992497

chrF++ score:

In [9]:
import sacrebleu

chrf_score = sacrebleu.corpus_chrf(target_texts.tolist(), [[reference] for reference in references])
chrf_score

chrF2 = 41.51